In [1]:
%run util.py

In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
# Loading JSON data
with open(data_folder + 'jam_ref_data_.json', 'r') as json_file:
    jam_ref_data = json.load(json_file)
    
with open(data_folder + 'jam_test_data_.json', 'r') as json_file:
    jam_test_data = json.load(json_file)

In [5]:
jam_ref_data_add_idx = {}

i = 0
for key in jam_ref_data.keys():
    data = {'uuid': str(key),
            'latitude': jam_ref_data[key]['latitude'],
            'longitude': jam_ref_data[key]['longitude'],
            'length': jam_ref_data[key]['length'],
            'numPts': float(jam_ref_data[key]['numPts']),
            'speed': jam_ref_data[key]['speed'],
            'startTime': jam_ref_data[key]['startTime']}
    jam_ref_data_add_idx[str(i)] = data
    i += 1

In [6]:
jam_test_data_add_idx = {}

i = 0
for key in jam_test_data.keys():
    data = {'uuid': str(key),
            'latitude': jam_test_data[key]['latitude'],
            'longitude': jam_test_data[key]['longitude'],
            'length': jam_test_data[key]['length'],
            'numPts': float(jam_test_data[key]['numPts']),
            'speed': jam_test_data[key]['speed'],
            'startTime': jam_test_data[key]['startTime']}
    jam_test_data_add_idx[str(i)] = data
    i += 1

In [7]:
jam_ref_data_add_idx['0'], jam_test_data_add_idx['0']

({'latitude': -71.06517799999999,
  'length': 322.0,
  'longitude': 42.362827333333335,
  'numPts': 12.0,
  'speed': 1.1277777777777775,
  'startTime': 71700.0,
  'uuid': '8651c397-a57b-3e6c-a123-090f813928d5'},
 {'latitude': -71.09464707692308,
  'length': 472.0,
  'longitude': 42.34562446153846,
  'numPts': 13.0,
  'speed': 3.0472222222222225,
  'startTime': 42300.0,
  'uuid': '48f951eb-a001-3a0e-b075-d261e82519ac'})

In [8]:
len(jam_ref_data_add_idx.keys()), len(jam_test_data_add_idx.keys())

(46758, 6795)

### Quantize the features by k-means clustering 

In [9]:
import numpy as np

loc_list = []
length_list = []
numPts_list = []
speed_list = []

ref_loc_list = []
ref_length_list = []
ref_numPts_list = []
ref_speed_list = []
for i in range(len(jam_ref_data_add_idx.keys())):
    ref_loc_list.append([jam_ref_data_add_idx[str(i)]['latitude'], jam_ref_data_add_idx[str(i)]['longitude']])
    loc_list.append([jam_ref_data_add_idx[str(i)]['latitude'], jam_ref_data_add_idx[str(i)]['longitude']])
    
    ref_length_list.append(jam_ref_data_add_idx[str(i)]['length'])
    length_list.append(jam_ref_data_add_idx[str(i)]['length'])
    
    ref_numPts_list.append(jam_ref_data_add_idx[str(i)]['numPts'])
    numPts_list.append(jam_ref_data_add_idx[str(i)]['numPts'])
    
    ref_speed_list.append(jam_ref_data_add_idx[str(i)]['speed'])
    speed_list.append(jam_ref_data_add_idx[str(i)]['speed'])


test_loc_list = []
test_length_list = []
test_numPts_list = []
test_speed_list = []
for i in range(len(jam_test_data_add_idx.keys())):
    test_loc_list.append([jam_test_data_add_idx[str(i)]['latitude'], jam_test_data_add_idx[str(i)]['longitude']])
    loc_list.append([jam_test_data_add_idx[str(i)]['latitude'], jam_test_data_add_idx[str(i)]['longitude']])
    
    test_length_list.append(jam_test_data_add_idx[str(i)]['length'])
    length_list.append(jam_test_data_add_idx[str(i)]['length'])
    
    test_numPts_list.append(jam_test_data_add_idx[str(i)]['numPts'])
    numPts_list.append(jam_test_data_add_idx[str(i)]['numPts'])
    
    test_speed_list.append(jam_test_data_add_idx[str(i)]['speed'])
    speed_list.append(jam_test_data_add_idx[str(i)]['speed'])

In [10]:
loc_data = np.array(loc_list)
length_data = np.array(length_list)
numPts_data = np.array(numPts_list)
speed_data = np.array(speed_list)

In [11]:
np.size(loc_data,0), np.size(loc_data,1), np.size(np.array(ref_loc_list),0), np.size(np.array(ref_loc_list),1),\
np.size(np.array(test_loc_list),0), np.size(np.array(test_loc_list),1), 46758 + 6795

(53553, 2, 46758, 2, 6795, 2, 53553)

### Quantization levels: loc_k = 3, length_k = 2, numPts_k = 2, speed_k = 2

In [12]:
from scipy.cluster.vq import kmeans2

loc_k = 3
loc_centroid, loc_label = kmeans2(loc_data, loc_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

length_k = 2
length_centroid, length_label = kmeans2(length_data, length_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

numPts_k = 2
numPts_centroid, numPts_label = kmeans2(numPts_data, numPts_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

speed_k = 2
speed_centroid, speed_label = kmeans2(speed_data, speed_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

In [13]:
loc_centroid, loc_label, length_centroid, length_label, numPts_centroid, numPts_label, speed_centroid, speed_label

(array([[-71.12261498,  42.33548072],
        [-71.06885777,  42.35130836],
        [-71.06898794,  42.29690178]]),
 array([1, 0, 2, ..., 0, 0, 0], dtype=int32),
 array([ 1338.28510998,   439.08775686]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int32),
 array([ 20.69022507,   7.5206511 ]),
 array([1, 1, 1, ..., 1, 0, 1], dtype=int32),
 array([ 4.65660471,  2.13121295]),
 array([1, 1, 1, ..., 1, 0, 1], dtype=int32))

In [14]:
ref_loc_label = loc_label[0:np.size(np.array(ref_loc_list),0)]
test_loc_label = loc_label[np.size(np.array(ref_loc_list),0):np.size(loc_data,0)]

ref_length_label = length_label[0:np.size(np.array(ref_length_list),0)]
test_length_label = length_label[np.size(np.array(ref_length_list),0):np.size(length_data,0)]

ref_numPts_label = numPts_label[0:np.size(np.array(ref_numPts_list),0)]
test_numPts_label = numPts_label[np.size(np.array(ref_numPts_list),0):np.size(numPts_data,0)]

ref_speed_label = speed_label[0:np.size(np.array(ref_speed_list),0)]
test_speed_label = speed_label[np.size(np.array(ref_speed_list),0):np.size(speed_data,0)]

In [15]:
len(ref_loc_label), len(test_loc_label), len(ref_length_label), len(test_length_label), \
len(ref_numPts_label), len(test_numPts_label), len(ref_speed_label), len(test_speed_label)

(46758, 6795, 46758, 6795, 46758, 6795, 46758, 6795)

In [16]:
quantized_jam_ref_data = {}

for i in range(len(jam_ref_data_add_idx.keys())):
    key = str(i)
    data = {'uuid': jam_ref_data_add_idx[key]['uuid'],
            'latitude': jam_ref_data_add_idx[key]['latitude'],
            'longitude': jam_ref_data_add_idx[key]['longitude'],
            'length': jam_ref_data_add_idx[key]['length'],
            'numPts': float(jam_ref_data_add_idx[key]['numPts']),
            'speed': jam_ref_data_add_idx[key]['speed'],
            'startTime': jam_ref_data_add_idx[key]['startTime'],
            'loc_label': float(ref_loc_label[i]),
            'length_label': float(ref_length_label[i]),
            'numPts_label': float(ref_numPts_label[i]),
            'speed_label': float(ref_speed_label[i])}
    quantized_jam_ref_data[key] = data
    
# Writing JSON data
with open(data_folder + 'quantized_jam_ref_data.json', 'w') as json_file:
    json.dump(quantized_jam_ref_data, json_file)

In [17]:
quantized_jam_test_data = {}

for i in range(len(jam_test_data_add_idx.keys())):
    key = str(i)
    data = {'uuid': jam_test_data_add_idx[key]['uuid'],
            'latitude': jam_test_data_add_idx[key]['latitude'],
            'longitude': jam_test_data_add_idx[key]['longitude'],
            'length': jam_test_data_add_idx[key]['length'],
            'numPts': float(jam_test_data_add_idx[key]['numPts']),
            'speed': jam_test_data_add_idx[key]['speed'],
            'startTime': jam_test_data_add_idx[key]['startTime'],
            'loc_label': float(test_loc_label[i]),
            'length_label': float(test_length_label[i]),
            'numPts_label': float(test_numPts_label[i]),
            'speed_label': float(test_speed_label[i])}
    quantized_jam_test_data[key] = data

# Writing JSON data
with open(data_folder + 'quantized_jam_test_data.json', 'w') as json_file:
    json.dump(quantized_jam_test_data, json_file)